# gathering thingiverse things

an effective scraping and caching strategy for gathering information on thingiverse.

each activity requires:
1. making requests
2. tidying requests
3. creating tasks to save salient data, create a file witht he missing columsn


1. gather the index of things from search
2. save the index to disk as parquet
3. read the index of things from disks
4. save the things to disc

the second request is solely to get the description information about the thing cause that is not included in the search response
despite the openapi schema saying that it should be


In [9]:
    import os, pandas, time, random, platformdirs, requests_cache
    from toolz.curried import *
    from pandas import DataFrame, Series, Index
    from pathlib import Path
    __import__("dotenv").load_dotenv()
    params = dict(access_token=os.environ["THINGIVERSE_ACCESS_TOKEN"])
    cache = platformdirs.user_cache_path("a11yhood") / "thingiverse"
    cache = Path("data") / "thingiverse"
    CachedSession = partial(requests_cache.CachedSession, backend="filesystem", serializer="json")
    search_session = CachedSession(cache / "search_responses")
    thing_session = CachedSession(cache / "thing_responses")

In [10]:
    def search_tags(term="assistive+technology", pages=None, per_page=30):
        """search thing"""
        page = 1
        while not pages or page <= pages:
            response = search_session.get(f"https://api.thingiverse.com/search/{term}", params | dict(per_page=per_page, page=page, type="thing"),)
            if pages is None:
                pages = (response.json()["total"] // per_page) + 1
            page += 1

In [12]:
    search_tags()

In [13]:
    results = (
        Series(search_session.cache.urls())
        .apply(search_session.get)
        .apply(operator.methodcaller("json"))
        .apply(lambda x: x.get("hits", []))
        .explode()
        .apply(Series)
        .set_index("public_url")
    )

In [ ]:
    for j, (i, row) in enumerate(results.dropna(subset="id").iterrows()):
        time.sleep(random.random()*2)
        thing_session.get(row.url, params=params)
        print(i, j)

https://www.thingiverse.com/thing:1852950 0
https://www.thingiverse.com/thing:1103082 1
https://www.thingiverse.com/thing:1871831 2
https://www.thingiverse.com/thing:2745606 3
https://www.thingiverse.com/thing:3057298 4
https://www.thingiverse.com/thing:2785602 5
https://www.thingiverse.com/thing:5936465 6
https://www.thingiverse.com/thing:242639 7
https://www.thingiverse.com/thing:2801872 8
https://www.thingiverse.com/thing:1104871 9
https://www.thingiverse.com/thing:1097099 10
https://www.thingiverse.com/thing:95818 11
https://www.thingiverse.com/thing:1105307 12
https://www.thingiverse.com/thing:3882190 13
https://www.thingiverse.com/thing:261462 14
https://www.thingiverse.com/thing:2754691 15
https://www.thingiverse.com/thing:16174 16
https://www.thingiverse.com/thing:4962946 17
https://www.thingiverse.com/thing:4478851 18
https://www.thingiverse.com/thing:1090461 19
https://www.thingiverse.com/thing:596966 20
https://www.thingiverse.com/thing:3541762 21
https://www.thingiverse.com

https://www.thingiverse.com/thing:1593407 1


https://www.thingiverse.com/thing:140519 2


https://www.thingiverse.com/thing:2596644 3


https://www.thingiverse.com/thing:1499148 4


...

https://www.thingiverse.com/thing:1104933 456


https://www.thingiverse.com/thing:2801934 457


https://www.thingiverse.com/thing:2440223 458


https://www.thingiverse.com/thing:1499148 459


https://www.thingiverse.com/thing:1908866 460


In [6]:
(
    Series(thing_session.cache.responses)
    .apply(operator.methodcaller("json"))
    .apply(Series)
)

,id,name,thumbnail,url,public_url,creator,added,modified,is_published,is_wip,...,is_derivative,ancestors,type_name,is_banned,is_comments_disabled,needs_moderation,is_decoy,zip_data,winner_name,winner_link
00fdfcd14237dd21,1098836,e-NABLE Phone Home Prototype 3,https://cdn.thingiverse.com/renders/00/a0/a6/2...,https://api.thingiverse.com/things/1098836,https://www.thingiverse.com/thing:1098836,"{'id': 173593, 'name': 'deltamodulator', 'firs...",2015-10-29T06:48:49+00:00,2015-10-29T06:48:49+00:00,1,1,...,True,"[{'id': 596966, 'name': 'Raptor Reloaded by e-...",Thing,False,False,0,0,{'files': [{'name': 'e-NABLE_Phone_Home_10.ino...,NaN,NaN
017ac16bf4936a98,2881400,3D Printed Arm Socket,https://cdn.thingiverse.com/renders/73/6e/58/d...,https://api.thingiverse.com/things/2881400,https://www.thingiverse.com/thing:2881400,"{'id': 184877, 'name': 'sujana', 'first_name':...",2018-04-27T04:06:29+00:00,2018-04-27T04:06:29+00:00,1,0,...,False,[],Thing,False,False,0,0,{'files': [{'name': 'Arm_Socket_C_27042018.stl...,NaN,NaN
0182e0eb0261935c,2061214,VESA Mount for Overhead Camera (IPevo HD Plus),https://cdn.thingiverse.com/renders/e2/9b/89/2...,https://api.thingiverse.com/things/2061214,https://www.thingiverse.com/thing:2061214,"{'id': 1276407, 'name': 'atmakers', 'first_nam...",2017-01-25T05:09:19+00:00,2017-01-25T05:09:19+00:00,1,0,...,False,[],Thing,False,False,0,0,{'files': [{'name': 'VESA_Mount_Overhead_Camer...,NaN,NaN
01de48b8131edf9c,3710570,Interatomic Bonds Tactile Model,https://cdn.thingiverse.com/renders/df/ba/57/9...,https://api.thingiverse.com/things/3710570,https://www.thingiverse.com/thing:3710570,"{'id': 2652477, 'name': 'ntaufscar', 'first_na...",2019-06-24T12:12:19+00:00,2019-06-24T12:12:19+00:00,1,1,...,False,[],Thing,False,False,0,0,"{'files': [{'name': 'ElectronExchange.stl', 'u...",NaN,NaN
0261cb5c628ba03f,6586072,Dolphin Flexi Fidget Toy - print in place,https://cdn.thingiverse.com/assets/43/bb/e6/78...,https://api.thingiverse.com/things/6586072,https://www.thingiverse.com/thing:6586072,"{'id': 5595608, 'name': 'timmytombstone1', 'fi...",2024-04-19T17:30:42+00:00,2024-04-19T17:30:42+00:00,1,0,...,False,[],Thing,False,False,0,0,"{'files': [{'name': 'dolphin.stl', 'url': 'htt...",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fc2f5a3939200c69,3486198,"Assistive Walker, Kaye walker.",https://cdn.thingiverse.com/renders/71/31/69/f...,https://api.thingiverse.com/things/3486198,https://www.thingiverse.com/thing:3486198,"{'id': 163526, 'name': 'hagridden', 'first_nam...",2019-03-12T04:17:20+00:00,2019-03-12T04:17:20+00:00,1,0,...,False,[],Thing,False,False,0,0,"{'files': [{'name': 'Walker_v1.stl', 'url': 'h...",NaN,NaN
fc434d2e733bd132,683496,Modified RIT Arm Parts,https://cdn.thingiverse.com/renders/33/44/e5/f...,https://api.thingiverse.com/things/683496,https://www.thingiverse.com/thing:683496,"{'id': 248357, 'name': 'masnart39', 'first_nam...",2015-02-15T13:01:20+00:00,2015-02-15T13:01:20+00:00,1,1,...,True,"[{'id': 604380, 'name': 'Modified RIT Arm Part...",Thing,False,False,0,0,{'files': [{'name': 'forearm-cup-modified.STL'...,NaN,NaN
fc7f60b1f323f996,2513492,I8-module_open°lights,https://cdn.thingiverse.com/renders/ac/5a/41/7...,https://api.thingiverse.com/things/2513492,https://www.thingiverse.com/thing:2513492,"{'id': 126601, 'name': 'FabLab-Berlin', 'first...",2017-09-02T17:33:01+00:00,2017-09-02T17:33:01+00:00,1,0,...,False,[],Thing,False,False,0,0,{'files': [{'name': 'Circuit_Diagram_Module_I....,NaN,NaN
fe780a7515816dee,2246592,Thumb Prosthesis,https://cdn.thingiverse.com/renders/14/7d/29/c...,https://api.thingiverse.com/things/2246592,https://www.thingiverse.com/thing:2246592,"{'id': 765168, 'name': 'GabrielaB', 'first_nam...",2017-04-13T14:25:48+00:00,2017-04-13T14:25:48+00:00,1,0,...,True,"[{'id': 261462, 'name': 'Cyborg Beast ', 'url'...",Thing,False,False,0,0,"{'files': [{'name': 'axis.STL', 'url': 'https:...",NaN,NaN
